In [ ]:
import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"
os.environ["NUMBA_DEBUG_CACHE"] = "1"

In [ ]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from quantfreedom.enums import DynamicOrderSettingsArrays
from quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
from numba import njit, types, typed
from numba.experimental import jitclass
import logging
from quantfreedom.base import final_bt_df_only
from quantfreedom.enums import *

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [ ]:
wins_and_losses_array_no_be = np.array([3,-])
win_loss = np.where(wins_and_losses_array_no_be < 0, 0, 1)
win_rate = round(np.count_nonzero(win_loss) / win_loss.size * 100, 2)